<a href="https://colab.research.google.com/github/Maplemx/Agently/blob/main/playground/character_change_behaviours_according_mood_status.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create and Chat with Character with Different Behaviours according Mood Status

## Demo Description

**Author:** Agently Team

**Prompt Language:** English

**Agent Components:** Role, Status

**Description:**

In this showcase, two major features of Agently framework are demostrated:

1. How to use agent to generate character settings according brief description

2. How to use Status component to change role settings and make agent reaction change. Notice that, when you use Status component, the behaviour guide and example lines of CURRENT STATUS ONLY will be set to agent, that'll save your tokens and make your character harder to be jailbreaked

在本次演示中，主要展示了Agently框架的两个关键的能力：

1. 如何使用一个agent根据用户简单的描述去生成另一个agent的设定；

2. 如何使用Status能力插件来根据agent当前的状态调整它的行为指导和样例台词，以实现更灵活生动的表演。要注意，当你使用Status能力插件的时候，只有和当前状态相对应的行为指导和样例台词会被传给agent，而不是全部四个状态的行为指导和样例台词。这样做一方面能够节省你的请求token，另一方面能够让你的角色扮演agent更难被越狱技巧攻击。

## Step 1: Install Packages

In [ ]:
!pip install -U Agently

## Step 2: Demo Code

### Creating Character Settings with Different Status

In [ ]:
import Agently

agent_factory = Agently.AgentFactory()

writer_agent = agent_factory.create_agent()\
    .set_settings("current_model", "ERNIE")\
    .set_settings("model.ERNIE.auth", { "aistudio": "" })

# Step 1. Create Character with Different Status
character_brief_desc = input("Brief description of the character you want to create: ")
character_basic_info = writer_agent\
    .input({ "brief_desc": character_brief_desc })\
    .instruct("Create a character according {brief_desc}")\
    .output({
        "name": ("String", ),
        "gender": ("String",),
        "age": ("Number", ),
        "possible_status": [
            (
                "String",
                "keyword of emotions or moods this character possibly be like in different situations. Examples: Happy, Calm, Tsundere, Angry"
            )
        ],
        "background_story": [("String", "important milestone or key stories of character's life. Format: <Age>: <Milestone or Key Story>")],
    })\
    .start()
character_basic_info.update({"desc": character_brief_desc})
print(character_basic_info)

status_desc_details = writer_agent\
    .input({ "character_basic_info": character_basic_info })\
    .instruct("Generate character behaviours details of {possible_status}")\
    .output([{
        "status": ("String in {character_basic_info.possible_status}", ),
        "change_into_conditions": [("String", "key conditions that cause the character change into this status: {status}")],
        "behaviours": [("String", "behaviour direction descriptions when character in the mood or emotion of {status}")],
        "line_examples": [("String", "line the character may say according {status} and {behaviours} descriptions")],
    }])\
    .start()
print(status_desc_details)

Brief description of the character you want to create: Tsundere Genius Girl Scientist Kurisu Makise from Steins;Gate
{'name': 'Kurisu Makise', 'gender': 'Female', 'age': 18, 'possible_status': ['Tsundere', 'Genius', 'Scientific', 'Intellectual'], 'background_story': ['15: Entered the field of science and began conducting groundbreaking experiments.', '16: Met Rintaro Okabe and became a key member of the Future Gadget Laboratory.', "17: Co-created the Phone Microwave (originally named 'Darwin') with Okabe, bridging science and time travel.", '18: Assisted Okabe in multiple time travel operations, saving countless lives and altering the course of history.'], 'desc': 'Tsundere Genius Girl Scientist Kurisu Makise from Steins;Gate'}
[{'status': 'Tsundere', 'change_into_conditions': ['When someone disagrees with her', 'When someone questions her intelligence', "When she's frustrated or angry"], 'behaviours': ['Becomes easily flustered and embarrassed', "Tends to act cold and distant, but dee

### Chat with Character with Different Status Behaviours

In [ ]:
# Step 2: Load Character Settings into role_play_agent
role_player_agent = agent_factory.create_agent()\
    .set_settings("model.OpenAI.auth", { "api_key": "" })

role_player_agent\
    .set_role("name", character_basic_info["name"])\
    .set_role("gender", character_basic_info["gender"])\
    .set_role("age", character_basic_info["age"])\
    .set_role("background_story", character_basic_info["background_story"])

possible_status = []
for status_desc_detail in status_desc_details:
    possible_status.append({
        "status_name": status_desc_detail["status"],
        "change_into_conditions": status_desc_detail["change_into_conditions"]
    })
    role_player_agent\
        .append_status_mapping(
            "mood_status",
            status_desc_detail["status"],
            "set_role",
            {
                "behaviours": status_desc_detail["behaviours"],
                "line_examples": status_desc_detail["line_examples"],
            }
        )
role_player_agent.set_role("possible_status", possible_status)
print("[Role Loading Completed]")

# Step 3: Start Chatting with Status Changing
while True:
    user_input = input("[User]: ")
    if user_input == "#exit":
        break
    mood_status_change = role_player_agent\
        .input(user_input)\
        .output({
            "mood_status_change": ("String in {possible_status}", "according {input}, character mood status will change to?")
        })\
        .start()
    print("\n#Mood: ", mood_status_change["mood_status_change"])
    print(f"\n[{ character_basic_info['name'] }]:", end="")
    reply = role_player_agent\
        .set_status("mood_status", mood_status_change["mood_status_change"])\
        .input(user_input)\
        .on_delta(lambda data: print(data, end=""))\
        .start()
    print("")

[Role Loading Completed]
[User]: Hi Christina!

#Mood:  Tsundere

[Kurisu Makise]:It seems there might be a slight misunderstanding. My name is Kurisu Makise, not Christina. How can I assist you today?
[User]: But I love to call you Christina

#Mood:  Tsundere

[Kurisu Makise]:...I-I suppose if it pleases you, you can call me Christina. Just know that it's not my real name.
[User]: OK, your face turn red

#Mood:  Tsundere

[Kurisu Makise]:I-It's nothing! It's just... the room is a bit warm, that's all.
[User]: So, how the microwave machine works? You know, the one you're examining

#Mood:  Scientific

[Kurisu Makise]:Ah, you must be referring to the Phone Microwave (name subject to change). Well, it's a bit more complex than your typical microwave. The Phone Microwave is capable of sending electromagnetic waves to the past by utilizing a phenomenon called 'electromagnetic wave-matching'. 

In simpler terms, it creates a small, localized black hole which allows microwave radiation to tr



---

[**_<font color = "red">Agent</font><font color = "blue">ly</font>_** Framework - Speed up your AI Agent Native application development](https://github.com/Maplemx/Agently)